# Fit Neg Binomial Auto-regressive model 

Taking inspiration from [GLM.jl](https://github.com/JuliaStats/GLM.jl/blob/master/src/negbinfit.jl#L68), we will:
+ Initialize `r` with Poisson regression fit
+ Perform block updates:
    - Fix $r$, fit negative binomial AR parameters ($\beta, \rho, \sigma^2$) for 10 iterations
    - Fix $\beta, \rho, \sigma^2$, fit $r$ using Newton for 10 iterations
    - Repeat until convergence

# Block updates

In [50]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices

function get_V(ρ, n)
    vec = zeros(n)
    vec[1] = 1.0
    for i in 2:n
        vec[i] = vec[i - 1] * ρ
    end
    V = ToeplitzMatrices.SymmetricToeplitz(vec)
    V
end

# simulation parameters
Random.seed!(12345)
N = 10000            # sample size
n = 50                # observations per subject
p = 3                # number of fixed effects, including intercept
σ2true = 0.1         # true σ2
ρtrue = 0.8          # true ρ
r = 100              # true dispersion parameter in NB model
β_true = ones(p)     # true beta
V = get_V(ρtrue, n)  # true variance
Γ = σ2true * V;      # true Gamma

In [51]:
# now simulate y
Random.seed!(12345)
d = NegativeBinomial()
link = LogLink()
gcs = Vector{NBCopulaARObs{Float64, typeof(d), typeof(link)}}(undef, N)
res = Vector{Float64}(undef, n)
for i in 1:N
    y = Vector{Float64}(undef, n)
    X = [ones(n) randn(n, p - 1)]
    η = X * β_true
    μ = exp.(η)
    prob = r ./ (μ .+ r)
    vecd = [NegativeBinomial(r, prob[i]) for i in 1:n]
    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
    rand(nonmixed_multivariate_dist, y, res) # this mutates y!!
    gcs[i] = NBCopulaARObs(y, X, d, link)
end
gcm = NBCopulaARModel(gcs);

In [52]:
initialize_model!(gcm)
@show gcm.β
@show gcm.r
@show gcm.σ2
@show gcm.ρ;

Initializing NegBin r to Poisson regression values


┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_ar.jl:31


initializing variance parameters in AR model using MM-Algorithm
gcm.β = [0.9985221395848457, 1.0008741860133186, 1.0002411197107366]
gcm.r = [59.781472157357555]
gcm.σ2 = [0.004287718097948753]
gcm.ρ = [1.0]


In [53]:
# fit negative binomial AR model
@time GLMCopula.fit!(gcm, maxBlockIter=30)

Block iter 1 r = 88.94, logl = -956098.6, tol = 956098.6031702517
Block iter 2 r = 93.27, logl = -955732.84, tol = 0.0003825564348494345
Block iter 3 r = 94.17, logl = -955725.3, tol = 7.890327652859225e-6
Block iter 4 r = 95.84, logl = -955726.79, tol = 1.5641611109831202e-6
Block iter 5 r = 94.48, logl = -955747.29, tol = 2.1448218159982597e-5
Block iter 6 r = 96.56, logl = -955749.19, tol = 1.9877854019396667e-6
Block iter 7 r = 94.79, logl = -955723.36, tol = 2.7030795726709242e-5
Block iter 8 r = 94.77, logl = -955722.14, tol = 1.2742877832636287e-6
Block iter 9 r = 97.15, logl = -955752.75, tol = 3.20231317420411e-5
Block iter 10 r = 94.63, logl = -955751.68, tol = 1.1181457682257198e-6
Block iter 11 r = 94.68, logl = -955723.56, tol = 2.941713753476216e-5
Block iter 12 r = 94.41, logl = -955747.55, tol = 2.5095639915891946e-5
140.222635 seconds (19.83 M allocations: 59.910 GiB, 5.07% gc time)


-955746.7517729481

In [54]:
println("estimated β = $(gcm.β); true β = $β_true")
println("estimated rho = $(gcm.ρ[1]); true rho = $ρtrue")
println("estimated σ2 = $(gcm.σ2[1]); true σ2 = $σ2true")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = [0.9966164588741423, 1.0021839970304707, 0.9997075318151586]; true β = [1.0, 1.0, 1.0]
estimated rho = 0.8830003263924501; true rho = 0.8
estimated σ2 = 0.04792968061162541; true σ2 = 0.1
estimated r = 94.38115334334883; true r = 100
